提取資料到array

In [7]:
import cv2
import numpy as np


In [8]:

y_train=[]
train_paths=[]


# Read text file containing image paths
with open('train.txt', 'r') as f:
    for line in f.readlines():  
    # print(line[:-1].split(' '))
        image_train = line[:-1].split(' ')

        train_paths.append(str(image_train[0]))
        train_labels = image_train[1]

        y_train.append(train_labels)



In [9]:
y_test=[]
test_paths=[]

# Read text file containing image paths
with open('test.txt', 'r') as f:
    from tqdm.notebook import tqdm

    for line in f.readlines():  
        # print(line[:-1].split(' '))
        image_test = line[:-1].split(' ') 

        test_paths.append(image_test[0])
        test_labels = image_test[1]
        y_test.append(test_labels)

In [10]:
from skimage.feature import hog
# 設定 HOG 參數
orientations = 9
pixels_per_cell = (8, 8)
cells_per_block = (3, 3)

def extract_histogram(image_path):
    
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    hog_features = hog(image, orientations=orientations, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block)
    
    return hog_features

In [11]:
from tqdm.notebook import tqdm

In [29]:
x_train=[]

with tqdm(total=len(train_paths[:10500])) as pbar:
    for path in train_paths[:10500]:
        try:
            features = extract_histogram(path)
            x_train.append(features)
        except Exception as e:
            pass
        finally:
            pbar.update()

  0%|          | 0/10500 [00:00<?, ?it/s]

In [34]:
print(len(y_train))
y_train_cut=y_train[:10500]
print(len(y_train_cut))


63325
10500


In [15]:
#測試 x_test
x_test=[]
with tqdm(total=len(test_paths)) as pbar:
    for path in test_paths:
        try:
            features = extract_histogram(path)
            x_test.append(features)
        except Exception as e:
            pass
        finally:
            pbar.update()

  0%|          | 0/450 [00:00<?, ?it/s]

In [35]:
print(x_test[0])
print(len(x_train))

[0.24870486 0.02180481 0.09653401 ... 0.         0.         0.05040989]
10500


In [37]:
x_test_cut=x_test[:35]
y_test_cut=y_test[:35]


In [38]:
# 缩小特徵向量的長度
length = 10000
for i in range(len(x_train)):
    features = x_train[i]
    features = np.resize(features, (length,))
    x_train[i] = features
    
for i in range(len(x_test_cut)):
    features = x_test_cut[i]
    features = np.resize(features, (length,))
    x_test_cut[i] = features

In [40]:
x_train_np = np.array(x_train)

In [41]:
x_test_np = np.array(x_test_cut)

In [42]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
# 切分資料集
# X_train, X_test, Y_train, Y_test = train_test_split(x_train_np, y_train, test_size=0.2)

# 建立 SVM 分類器
svm = SVC(kernel='linear')

# 訓練分類器
svm.fit(x_train_np, y_train_cut)

# 進行預測
y_pred = svm.predict(x_test_np)

# 評估分類器的準確率
accuracy = svm.score(x_test_np, y_test_cut)
print("Accuracy:", accuracy)

Accuracy: 0.17142857142857143


In [43]:
from sklearn.linear_model import Perceptron
clf = Perceptron()
clf.fit(x_train_np, y_train_cut)

Perceptron()

In [44]:
y_pred_perceptron = clf.predict(x_test_np)

In [45]:
accuracy_perceptron = clf.score(x_test_np, y_test_cut)
print("Accuracy:", accuracy_perceptron)

Accuracy: 0.14285714285714285
